In [ ]:
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_ollama import ChatOllama
from langchain.tools import tool
import random
import pprint


In [ ]:
# Simulated book database
book_database = {
    "Thinking, Fast and Slow": {"creativity": 2, "accuracy": 5, "knowledge": 5},
    "The Alchemist": {"creativity": 5, "accuracy": 2, "knowledge": 3},
    "Sapiens": {"creativity": 4, "accuracy": 4, "knowledge": 5},
    "1984": {"creativity": 3, "accuracy": 4, "knowledge": 4},
    "Harry Potter": {"creativity": 5, "accuracy": 3, "knowledge": 3},
}

# Initial user state
state = {
    "preferences": {"creativity": 8, "accuracy": 4, "knowledge": 6},
    "recommended_book": "None",
    "feedback": "0.0",
    "weights": {"creativity": 5.0, "accuracy": 5.0, "knowledge": 5.0},
}

In [ ]:
from langchain.tools import tool

@tool
def recommend_book() -> str:
    """Recommend the best book based on weighted user preferences."""
    global state
    weights = state["weights"]
    scores = {}
    for book, traits in book_database.items():
        score = sum(weights.get(k, 0) * traits.get(k, 0) for k in traits)
        scores[book] = score
    best_book = max(scores, key=scores.get)
    state["recommended_book"] = best_book
    return f"Recommended book: {best_book} based on weights {weights}"

@tool
def simulate_feedback() -> str:
    """Simulate user feedback based on how well the recommended book aligns with preferences."""
    global state
    book = state["recommended_book"]
    prefs = state["preferences"]
    traits = book_database.get(book, {})
    alignment = sum(
        1 if traits.get(k, 0) >= prefs.get(k, 5) else -1
        for k in prefs
    )
    feedback = "good" if alignment >= 0 else "bad"
    state["feedback"] = feedback
    return f"User gave '{book}' a feedback: {feedback}"

@tool
def update_weights() -> str:
    """Update weights based on feedback to improve future recommendations."""
    global state
    feedback = state["feedback"]
    book = state["recommended_book"]
    traits = book_database[book]
    adjustment = 1 if feedback == "good" else -1

    for k in state["weights"]:
        delta = 0.1 * adjustment * traits.get(k, 0)
        state["weights"][k] += delta
        state["weights"][k] = max(0.0, min(10.0, state["weights"][k]))
    return f"Updated weights: {state['weights']}"

In [ ]:
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_ollama import ChatOllama

# Load local Ollama LLM
llm = ChatOllama(model="deepseek-r1:1.5b", temperature=0.7)

# Define tools with proper name and description
tools = [
    Tool.from_function(
        func=recommend_book,
        name="RecommendBook",
        description="Recommend a book based on user preferences."
    ),
    Tool.from_function(
        func=simulate_feedback,
        name="SimulateFeedback",
        description="Simulate user feedback for a given recommendation."
    ),
    Tool.from_function(
        func=update_weights,
        name="UpdateWeights",
        description="Update the preference weights based on feedback."
    ),
]

# Create the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
# Run 3 rounds of book recommendation
for i in range(3):
    print(f"\n=== Round {i+1} ===")

    # Step 1: Prompt LLM to choose a book title
    prompt = (
        f"User preferences: {state['preferences']}.\n"
        f"Choose ONE exact book title from: {', '.join(book_database.keys())}.\n"
        "Reply with ONLY the book title. No explanations. No extra words."
    )
    response = llm.invoke(prompt).content.strip()

    # Step 2: Match valid title
    matched_title = next(
        (title for title in book_database if title.lower() in response.lower()),
        None
    )
    if not matched_title:
        print("[Error] Invalid book title returned by LLM:", response)
        continue

    state["recommended_book"] = matched_title

    # Step 3: Short explanation
    traits = book_database[matched_title]
    explain_prompt = (
        f"User prefs: {state['preferences']}. Book traits: {traits}. "
        f"Why is '{matched_title}' a good match? Reply with ONE short sentence ONLY. No <think>. No reasoning steps."
    )
    explanation = llm.invoke(explain_prompt).content.strip()

    # Step 4: Print result
    print(f"[Final Recommendation] {matched_title}")
    print(f"[Reason] {explanation}")